# Data Profiling

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport

In [70]:
src = pd.read_csv('data/00_dataset_cleansed.csv')
print(src.shape)
src.info()

(316822, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316822 entries, 0 to 316821
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    316822 non-null  int64  
 1   loan_amnt             316822 non-null  int64  
 2   term                  316822 non-null  object 
 3   int_rate              316822 non-null  float64
 4   installment           316822 non-null  float64
 5   grade                 316822 non-null  object 
 6   sub_grade             316822 non-null  object 
 7   emp_title             298748 non-null  object 
 8   emp_length            302489 non-null  object 
 9   home_ownership        316822 non-null  object 
 10  annual_inc            316822 non-null  float64
 11  verification_status   316822 non-null  object 
 12  loan_status           316822 non-null  object 
 13  purpose               316822 non-null  object 
 14  title                 315430 non-null  

# Original Data Profiling

In [68]:
profile = ProfileReport(src)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Python\anaconda3\envs\corgi\lib\site-packages\scipy\stats\stats.py:4812: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [122]:
src.emp_length.value_counts(dropna=False)

10+ years    101131
2 years       28732
3 years       25355
< 1 year      25255
5 years       21284
1 year        20676
4 years       19233
6 years       16708
7 years       16544
8 years       15412
NaN           14332
9 years       12159
Name: emp_length, dtype: int64

In [125]:
src.emp_length.fillna('NONE').value_counts(dropna=False)

10+ years    101131
2 years       28732
3 years       25355
< 1 year      25255
5 years       21284
1 year        20676
4 years       19233
6 years       16708
7 years       16544
8 years       15412
NONE          14332
9 years       12159
Name: emp_length, dtype: int64

# Data Quality Check and EDA

## Removing rows

Annual Income should be required for loan application.  
There's only one row with `annual_inc == 0`

In [71]:
(src.annual_inc == 0).sum()

1

In [76]:
src = src[src.annual_inc > 0]

# Experimental Feature

## Feature Processing (Experiments)

In [126]:
def preprocess_dtypes(df):
    data = df.copy()
    
    for col in ['loan_amnt','revol_bal']:
        data[col] = data[col].astype(float)
        
    # emp_length
    emp_length_levels = ['NONE', '< 1 year', '1 year', '2 years', '3 years', '4 years', 
                        '5 years', '6 years', '7 years', '8 years', '9 years', '10+ years']
    data.emp_length = data.emp_length.fillna('NONE').astype(pd.CategoricalDtype(categories=emp_length_levels, ordered=True))
    
    # grade
    grade_levels = ['A','B','C','D','E','F','G']
    data.grade = data.grade.astype(pd.CategoricalDtype(categories=grade_levels, ordered=True))
    
    # sub grade
    sub_grade_levels = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 
                        'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 
                        'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 
                        'G1', 'G2', 'G3', 'G4', 'G5']
    data.sub_grade = data.sub_grade.astype(pd.CategoricalDtype(categories=sub_grade_levels, ordered=True))
    
    # term
    term_levels = ['36 months', '60 months']
    data.term = data.term.str.strip().astype(pd.CategoricalDtype(categories=term_levels, ordered=True))
    
    # loan status
    loan_status_levels = ['Charged Off', 'Fully Paid']
    data.loan_status = data.loan_status.astype(pd.CategoricalDtype(categories=loan_status_levels))

    # home ownership
    data.home_ownership = data.home_ownership.replace({'NONE':'OTHER','ANY':'OTHER'}).astype('category')
    
    # purpose, intiial_list_staus, application_type
    data.purpose = data.purpose.astype('category')
    data.initial_list_status = data.initial_list_status.astype('category')
    data.application_type = data.application_type.astype('category')
    
    # earlies_cr_line : datetime
    data.earliest_cr_line = pd.to_datetime(data.earliest_cr_line)
    
    return data

def featurize(df):
    data = df.copy()
    data['inc_not_verified'] = data.verification_status.eq('Not Verified')
    data['earliest_cr_year'] = data.earliest_cr_line.dt.year.astype(int)
    data['revol_util_missing'] = data.revol_util.isna()
    data['mort_acc_missing'] = data.mort_acc.isna()
    data['pub_rec_bankrupt_missing'] = data.pub_rec_bankruptcies.isna()
    data['zip'] = data.address.str.split('[\s,]', regex=True).apply(lambda x: x[-1]).astype('category')
    data['state'] = data.address.str.split('[\s,]', regex=True).apply(lambda x: x[-2]).astype('category')
    data['installment_inc_ratio'] = data['installment'] / data['annual_inc'] * 12
    return data

def drop_features(df):
    data = df.copy()
    drop_cols = ['id','emp_title','earliest_cr_line','address','title']
    data = data.drop(columns=drop_cols)    
    return data


# # impute missing values

# impute_features = ['revol_util','mort_acc','pub_rec_bankruptcies']
# for col in impute_features:
#     df[col] = df[col].fillna(df[col].median())
#     # df[col] = df[col].fillna(0)
    
# df.mort_acc = df.mort_acc.astype(int)
# df.pub_rec_bankruptcies = df.pub_rec_bankruptcies.astype(int)


# # one hot
# nominal_features = ['home_ownership', 'purpose', 
#                 'initial_list_status', 'application_type','zip'] # 'state'
# df = pd.get_dummies(df, columns=nominal_features)

In [127]:
df = preprocess_dtypes(src)
df = featurize(df)
df = drop_features(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316821 entries, 0 to 316821
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   loan_amnt                 316821 non-null  float64 
 1   term                      316821 non-null  category
 2   int_rate                  316821 non-null  float64 
 3   installment               316821 non-null  float64 
 4   grade                     316821 non-null  category
 5   sub_grade                 316821 non-null  category
 6   emp_length                316821 non-null  category
 7   home_ownership            316821 non-null  category
 8   annual_inc                316821 non-null  float64 
 9   verification_status       316821 non-null  object  
 10  loan_status               316821 non-null  category
 11  purpose                   316821 non-null  category
 12  dti                       316821 non-null  float64 
 13  open_acc                  316

## Processced Data Profiling (Experiments)

In [128]:
profile = ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Python\anaconda3\envs\corgi\lib\site-packages\scipy\stats\stats.py:4812: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
C:\Python\anaconda3\envs\corgi\lib\site-packages\scipy\stats\stats.py:4814: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(var) / np.sqrt(2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [112]:
src.to_csv('data/01_cleansed.csv', index=False)

In [129]:
df.isna().mean()

loan_amnt                   0.000000
term                        0.000000
int_rate                    0.000000
installment                 0.000000
grade                       0.000000
sub_grade                   0.000000
emp_length                  0.000000
home_ownership              0.000000
annual_inc                  0.000000
verification_status         0.000000
loan_status                 0.000000
purpose                     0.000000
dti                         0.000000
open_acc                    0.000000
pub_rec                     0.000000
revol_bal                   0.000000
revol_util                  0.000698
total_acc                   0.000000
initial_list_status         0.000000
application_type            0.000000
mort_acc                    0.097162
pub_rec_bankruptcies        0.001367
inc_not_verified            0.000000
earliest_cr_year            0.000000
revol_util_missing          0.000000
mort_acc_missing            0.000000
pub_rec_bankrupt_missing    0.000000
z